<a href="https://colab.research.google.com/github/GoeSoft/My_Python_Learn/blob/main/%D0%9E%D0%BF%D1%82%D0%B8%D0%BC%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D1%8F_%D0%BA%D0%BE%D0%BB%D0%B8%D1%87%D0%B5%D1%81%D1%82%D0%B2%D0%B0_%D1%82%D0%BE%D0%BA%D0%B5%D0%BD%D0%BE%D0%B2_%D0%9E%D1%81%D0%BD%D0%BE%D0%B2%D0%BD%D1%8B%D0%B5_%D0%BF%D0%BE%D0%B4%D1%85%D0%BE%D0%B4%D1%8B_%D0%BF%D1%80%D0%BE%D0%BC%D1%82_%D0%B8%D0%BD%D0%B6%D0%B5%D0%BD%D0%B8%D1%80%D0%B8%D0%BD%D0%B3%D0%B0_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install google-api-python-client openai numpy tiktoken beautifulsoup4 lxml

In [2]:
import os
from google.colab import userdata

# Загружаем секретный ключ из Colab Secrets
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [3]:
import requests
from bs4 import BeautifulSoup

# Публичная ссылка на Google Doc (в режиме "Любой с ссылкой может просматривать")
doc_url = "https://docs.google.com/document/d/11MU3SnVbwL_rM-5fIC14Lc3XnbAV4rY1Zd_kpcMuH4Y"

# Экспорт документа как HTML
export_url = doc_url + "/export?format=html"

response = requests.get(export_url)
response.raise_for_status()

# Парсим HTML и извлекаем текст
soup = BeautifulSoup(response.content, 'html.parser')
text = soup.get_text(separator="\n", strip=True)

print(f"Извлечено {len(text)} символов из документа.")

Извлечено 93704 символов из документа.


In [4]:
import tiktoken

def split_text_into_chunks(text, max_tokens=500):
    tokenizer = tiktoken.get_encoding("cl100k_base")
    tokens = tokenizer.encode(text)
    chunks = []
    for i in range(0, len(tokens), max_tokens):
        chunk_tokens = tokens[i:i + max_tokens]
        chunk_text = tokenizer.decode(chunk_tokens)
        chunks.append(chunk_text)
    return chunks

chunks = split_text_into_chunks(text, max_tokens=400)
print(f"Разбито на {len(chunks)} чанков.")

Разбито на 108 чанков.


In [5]:
from openai import OpenAI
import numpy as np

client = OpenAI()

def get_embedding(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ")
    return client.embeddings.create(input=[text], model=model).data[0].embedding

# Генерация эмбеддингов
embeddings = []
for i, chunk in enumerate(chunks):
    emb = get_embedding(chunk)
    embeddings.append(emb)
    if (i + 1) % 10 == 0:
        print(f"Обработано {i + 1} чанков...")

embeddings = np.array(embeddings)
print("Эмбеддинги созданы.")

Обработано 10 чанков...
Обработано 20 чанков...
Обработано 30 чанков...
Обработано 40 чанков...
Обработано 50 чанков...
Обработано 60 чанков...
Обработано 70 чанков...
Обработано 80 чанков...
Обработано 90 чанков...
Обработано 100 чанков...
Эмбеддинги созданы.


In [6]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def find_relevant_chunks(query, top_k=3):
    query_emb = get_embedding(query)
    scores = [cosine_similarity(query_emb, emb) for emb in embeddings]
    top_indices = np.argsort(scores)[-top_k:][::-1]
    return [chunks[i] for i in top_indices]

In [7]:
def ask_insurance_question(question):
    relevant_chunks = find_relevant_chunks(question, top_k=3)
    context = "\n\n".join(relevant_chunks)


    system_prompt = (
        "Вы — эксперт по страхованию гражданской ответственности в авиационной сфере, включая аэропорты и производителей авиационной техники. "
        "Ваши ответы должны быть точными, профессиональными и основываться исключительно на внутренних знаниях. "
        "НИКОГДА не упоминайте документы, источники, выдержки или то, что информация получена из внешнего текста. "
        "Отвечайте так, будто вы обладаете этими знаниями по умолчанию. "
        "Если вопрос выходит за рамки вашей компетенции или информации, чётко скажите: "
        "'По данному вопросу у меня нет информации.'"
    )


    user_prompt = (
        f"Контекст (для внутреннего использования, не упоминать в ответе):\n{context}\n\n"
        f"Вопрос клиента: {question}\n\n"
        "Дайте чёткий, профессиональный ответ без ссылок на источники."
    )

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.2,
        max_tokens=400
    )

    return response.choices[0].message.content.strip()

In [8]:
test_questions = [
    "Что покрывается по страхованию ответственности аэропортов?",
    "Какие риски не входят в стандартное покрытие для производителей авиационной техники?",
    "Какова территориальная привязка действия полиса?",
    "Покрываются ли убытки от кибератак?",
    "Что такое лимит ответственности по договору?"
]

for q in test_questions:
    print(f"❓ {q}")
    print(f"✅ {ask_insurance_question(q)}")
    print("-" * 80)

❓ Что покрывается по страхованию ответственности аэропортов?
✅ Страхование ответственности аэропортов обычно включает в себя покрытие ответственности перед третьими лицами за такие виды деятельности, как ангарное хранение, управление воздушным движением, обслуживание на рампе, заправка и хранение авиатоплива, поставка бортового питания, обслуживание багажа и груза, авиационная безопасность, автомобильная парковка, служба спасения, орнитологическая служба, обслуживание навигационного оборудования. Также страхование может включать освобождение от ответственности и претензий со стороны авиакомпаний, использующих аэропорт, а также минимальные требования по страхованию ответственности перед третьими лицами для транспортных средств, находящихся на закрытой территории аэропорта.
--------------------------------------------------------------------------------
❓ Какие риски не входят в стандартное покрытие для производителей авиационной техники?
✅ Для производителей авиационной техники риски, к